In [1]:
!pip install segmentation-models-pytorch
!pip install albumentations
!pip install torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 28.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_6

In [2]:
import os
import numpy as np
import pandas as pd
import cv2
from torch.utils.data import Dataset, DataLoader
from albumentations import Compose, Normalize, Resize, CenterCrop
from albumentations.pytorch import ToTensorV2
import torch
import torch.nn as nn
import torch.optim as optim
from segmentation_models_pytorch import Unet
from segmentation_models_pytorch.encoders import get_preprocessing_fn

In [3]:
# 구글 드라이브를 마운트합니다.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Load the data
train_df = pd.read_csv('/content/drive/MyDrive/클라우드컴퓨팅 텀프로젝트/dataset/TrainData.csv')
valid_df = pd.read_csv('/content/drive/MyDrive/클라우드컴퓨팅 텀프로젝트/dataset/ValidData.csv')
test_df = pd.read_csv('/content/drive/MyDrive/클라우드컴퓨팅 텀프로젝트/dataset/TestData.csv')

# Utility functions
def rle_decode(mask_rle, shape=(256, 256)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = list(map(int, mask_rle.split()))
    starts, lengths = s[0::2], s[1::2]
    starts = np.asarray(starts) - 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape).T

class LesionDataset(Dataset):
    def __init__(self, df, image_dir, transforms=None):
        self.df = df
        self.image_dir = image_dir
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_name = self.df.iloc[idx]['Image']
        img_path = os.path.join(self.image_dir, img_name + '.jpg')

        if not os.path.exists(img_path):
            raise FileNotFoundError(f"Image file {img_path} not found")

        image = cv2.imread(img_path)
        if image is None:
            raise ValueError(f"Failed to load image file {img_path}")

        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask_rle = self.df.iloc[idx]['EncodedPixels']
        mask = rle_decode(mask_rle, shape=image.shape[:2])

        if self.transforms:
            augmented = self.transforms(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        mask = np.expand_dims(mask, axis=0)  # Add channel dimension to the mask
        return image, mask


In [5]:
# Data transformations
transforms = Compose([
    CenterCrop(300, 300),
    Resize(224, 224),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
], additional_targets={'mask': 'mask'}, is_check_shapes=False)


# Create datasets and dataloaders
train_dataset = LesionDataset(train_df, '/content/drive/MyDrive/클라우드컴퓨팅 텀프로젝트/dataset/images/', transforms=transforms)
valid_dataset = LesionDataset(valid_df, '/content/drive/MyDrive/클라우드컴퓨팅 텀프로젝트/dataset/images/', transforms=transforms)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8, shuffle=False)

In [6]:
# Model setup
ENCODER = 'resnet34'
ENCODER_WEIGHTS = 'imagenet'
CLASSES = ['lesion']
ACTIVATION = 'sigmoid' # could be None for logits or 'softmax2d' for multiclass segmentation

model = Unet(encoder_name=ENCODER,
             encoder_weights=ENCODER_WEIGHTS,
             classes=len(CLASSES),
             activation=ACTIVATION)

preprocess_input = get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

# Training setup
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = model.to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 172MB/s]


In [7]:
def iou_pytorch(outputs: torch.Tensor, labels: torch.Tensor, threshold: float = 0.5):
    outputs = outputs > threshold
    labels = labels > threshold  # Convert labels to boolean as well
    intersection = (outputs & labels).float().sum((1, 2, 3))
    union = (outputs | labels).float().sum((1, 2, 3))
    iou = (intersection + 1e-6) / (union + 1e-6)
    return iou.mean().item()

def dice_coef(outputs: torch.Tensor, labels: torch.Tensor, threshold: float = 0.5):
    outputs = outputs > threshold
    labels = labels > threshold  # Convert labels to boolean as well
    intersection = (outputs & labels).float().sum((1, 2, 3))
    dice = (2. * intersection + 1e-6) / (outputs.float().sum((1, 2, 3)) + labels.float().sum((1, 2, 3)) + 1e-6)
    return dice.mean().item()

In [ ]:
# Training loop
num_epochs = 10
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    train_iou = 0
    train_dice = 0

    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.float().to(device)  # Convert masks to float

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        outputs = torch.sigmoid(outputs)  # Apply sigmoid activation for thresholding
        train_iou += iou_pytorch(outputs, masks)
        train_dice += dice_coef(outputs, masks)

    train_iou /= len(train_loader)
    train_dice /= len(train_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader)}, IoU: {train_iou}, Dice: {train_dice}")

    model.eval()
    val_loss = 0
    val_iou = 0
    val_dice = 0
    with torch.no_grad():
        for images, masks in valid_loader:
            images = images.to(device)
            masks = masks.float().to(device)  # Convert masks to float
            outputs = model(images)
            loss = criterion(outputs, masks)
            val_loss += loss.item()

            outputs = torch.sigmoid(outputs)  # Apply sigmoid activation for thresholding
            val_iou += iou_pytorch(outputs, masks)
            val_dice += dice_coef(outputs, masks)

    val_iou /= len(valid_loader)
    val_dice /= len(valid_loader)

    print(f"Validation Loss: {val_loss/len(valid_loader)}, IoU: {val_iou}, Dice: {val_dice}")

    # Save the model if validation loss has decreased
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '/content/drive/MyDrive/클라우드컴퓨팅 텀프로젝트/model/best_model.pth')
        print(f"Model saved at epoch {epoch+1}")

Epoch 1/10, Loss: 0.5371626620878003, IoU: 0.5896220497801871, Dice: 0.6965784303501933
Validation Loss: 0.5078391194343567, IoU: 0.5842976943254471, Dice: 0.6917821450233459
Model saved at epoch 1
Epoch 2/10, Loss: 0.5053838477163257, IoU: 0.5898029260828109, Dice: 0.6966990130747149
Validation Loss: 0.5044890949726105, IoU: 0.5842976943254471, Dice: 0.6917821450233459
Model saved at epoch 2
Epoch 3/10, Loss: 0.49886200786707646, IoU: 0.590485998048278, Dice: 0.6972839575327799
Validation Loss: 0.5021552278995514, IoU: 0.5842976943254471, Dice: 0.6917821450233459
Model saved at epoch 3
Epoch 4/10, Loss: 0.4955671351707862, IoU: 0.5922187478272263, Dice: 0.6986568016027976
Validation Loss: 0.4993251483440399, IoU: 0.5842977339029312, Dice: 0.6917822065353394
Model saved at epoch 4


In [ ]:
# 저장된 모델 가중치 로드
model.load_state_dict(torch.load('/content/drive/MyDrive/클라우드컴퓨팅 텀프로젝트/model/best_model.pth'))

<All keys matched successfully>

In [ ]:
# Training loop
num_epochs = 50
best_val_loss = float('inf')

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0
    train_iou = 0
    train_dice = 0

    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.float().to(device)  # Convert masks to float

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

        outputs = torch.sigmoid(outputs)  # Apply sigmoid activation for thresholding
        train_iou += iou_pytorch(outputs, masks)
        train_dice += dice_coef(outputs, masks)

    train_iou /= len(train_loader)
    train_dice /= len(train_loader)

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader)}, IoU: {train_iou}, Dice: {train_dice}")

    model.eval()
    val_loss = 0
    val_iou = 0
    val_dice = 0
    with torch.no_grad():
        for images, masks in valid_loader:
            images = images.to(device)
            masks = masks.float().to(device)  # Convert masks to float
            outputs = model(images)
            loss = criterion(outputs, masks)
            val_loss += loss.item()

            outputs = torch.sigmoid(outputs)  # Apply sigmoid activation for thresholding
            val_iou += iou_pytorch(outputs, masks)
            val_dice += dice_coef(outputs, masks)

    val_iou /= len(valid_loader)
    val_dice /= len(valid_loader)

    print(f"Validation Loss: {val_loss/len(valid_loader)}, IoU: {val_iou}, Dice: {val_dice}")

    # Save the model if validation loss has decreased
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '/content/drive/MyDrive/클라우드컴퓨팅 텀프로젝트/model/best_model.pth')
        print(f"Model saved at epoch {epoch+1}")

Epoch 1/50, Loss: 0.4950617585412994, IoU: 0.5993344316820423, Dice: 0.7044937282265303
Validation Loss: 0.500605719089508, IoU: 0.5843105634450912, Dice: 0.6917952637672424
Model saved at epoch 1
Epoch 2/50, Loss: 0.4914667147719218, IoU: 0.6117084709529629, Dice: 0.7144371572130931
Validation Loss: 0.49985160112380983, IoU: 0.5898717188835144, Dice: 0.6963595263957977
Model saved at epoch 2
Epoch 3/50, Loss: 0.4897690178629405, IoU: 0.6213400838320364, Dice: 0.7219486536974916
Validation Loss: 0.49929597520828245, IoU: 0.5947220466136932, Dice: 0.7005916571617127
Model saved at epoch 3
Epoch 4/50, Loss: 0.48704598528777293, IoU: 0.6334767414781148, Dice: 0.7312306762039067
Validation Loss: 0.4972623932361603, IoU: 0.6513597254753113, Dice: 0.7491750526428222
Model saved at epoch 4
Epoch 5/50, Loss: 0.48685904920933964, IoU: 0.6413610852704552, Dice: 0.7383057777723391
Validation Loss: 0.4982065622806549, IoU: 0.5887878535985946, Dice: 0.6956394240856171
Epoch 6/50, Loss: 0.4875860564

In [8]:
# 저장된 모델 가중치 로드
model.load_state_dict(torch.load('/content/drive/MyDrive/클라우드컴퓨팅 텀프로젝트/model/best_model.pth'))

<All keys matched successfully>

In [9]:
test_dataset = LesionDataset(test_df, '/content/drive/MyDrive/클라우드컴퓨팅 텀프로젝트/dataset/images/', transforms=transforms)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [10]:
model.eval()
test_loss = 0
test_iou = 0
test_dice = 0
with torch.no_grad():
    for images, masks in test_loader:
        images = images.to(device)
        masks = masks.float().to(device)  # Convert masks to float
        outputs = model(images)
        loss = criterion(outputs, masks)
        test_loss += loss.item()
        outputs = torch.sigmoid(outputs)  # Apply sigmoid activation for thresholding
        test_iou += iou_pytorch(outputs, masks)
        test_dice += dice_coef(outputs, masks)
test_iou /= len(test_loader)
test_dice /= len(test_loader)
print(f"Test Loss: {test_loss/len(test_loader)}, IoU: {test_iou}, Dice: {test_dice}")


Test Loss: 0.49455246707749745, IoU: 0.851066068524406, Dice: 0.9112102843466259
